In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '007'

BASE_DIR       = '../'
IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.001

In [3]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [9]:
model_xy = get_model()
history_xy = model_xy.fit(X_train_xy, y_train_xy, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 14s 378ms/step - loss: 2.9029 - accuracy: 0.1179
Epoch 2/7
30/30 [==============================] - 11s 355ms/step - loss: 1.6440 - accuracy: 0.4956
Epoch 3/7
30/30 [==============================] - 11s 367ms/step - loss: 1.2109 - accuracy: 0.6328
Epoch 4/7
30/30 [==============================] - 11s 362ms/step - loss: 1.0465 - accuracy: 0.6720
Epoch 5/7
30/30 [==============================] - 11s 352ms/step - loss: 0.8746 - accuracy: 0.7317
Epoch 6/7
30/30 [==============================] - 10s 343ms/step - loss: 0.7024 - accuracy: 0.7888
Epoch 7/7
30/30 [==============================] - 10s 331ms/step - loss: 0.6258 - accuracy: 0.8211


In [10]:
prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
y_pred_xy = prob_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.33      0.90      0.49        10
         2.0       0.67      0.40      0.50        10
         3.0       0.00      0.00      0.00        10
         4.0       0.40      0.20      0.27        10
         5.0       0.86      0.60      0.71        10
         6.0       0.67      0.40      0.50        10
         7.0       0.50      0.70      0.58        10
         8.0       0.83      1.00      0.91        10
         9.0       0.00      0.00      0.00        10
        10.0       0.54      0.70      0.61        10
        11.0       0.75      0.30      0.43        10
        12.0       0.33      0.30      0.32        10
        13.0       0.88      0.70      0.78        10
        14.0       1.00      0.90      0.95        10
        15.0       0.07      0.10      0.08        10

    accuracy                           0.51       160
   macro avg       0.55   

In [11]:
model_yz = get_model()
history_yz = model_yz.fit(X_train_yz, y_train_yz, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 333ms/step - loss: 3.2921 - accuracy: 0.1558
Epoch 2/7
30/30 [==============================] - 10s 332ms/step - loss: 1.7957 - accuracy: 0.4329
Epoch 3/7
30/30 [==============================] - 10s 330ms/step - loss: 1.1850 - accuracy: 0.5949
Epoch 4/7
30/30 [==============================] - 10s 327ms/step - loss: 0.9809 - accuracy: 0.6765
Epoch 5/7
30/30 [==============================] - 10s 331ms/step - loss: 0.7918 - accuracy: 0.7372
Epoch 6/7
30/30 [==============================] - 10s 326ms/step - loss: 0.7026 - accuracy: 0.7895
Epoch 7/7
30/30 [==============================] - 10s 331ms/step - loss: 0.6022 - accuracy: 0.8110


In [12]:
prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
y_pred_yz = prob_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87        10
         1.0       0.50      1.00      0.67        10
         2.0       0.62      0.80      0.70        10
         3.0       0.67      0.20      0.31        10
         4.0       0.33      0.10      0.15        10
         5.0       0.50      0.10      0.17        10
         6.0       0.57      0.80      0.67        10
         7.0       0.80      0.40      0.53        10
         8.0       0.75      0.90      0.82        10
         9.0       0.80      0.40      0.53        10
        10.0       0.44      0.70      0.54        10
        11.0       0.55      0.60      0.57        10
        12.0       0.38      0.50      0.43        10
        13.0       0.54      0.70      0.61        10
        14.0       0.70      0.70      0.70        10
        15.0       0.57      0.40      0.47        10

    accuracy                           0.58       160
   macro avg       0.59   

In [13]:
model_zx = get_model()
history_zx = model_zx.fit(X_train_zx, y_train_zx, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 331ms/step - loss: 3.5341 - accuracy: 0.2047
Epoch 2/7
30/30 [==============================] - 10s 329ms/step - loss: 2.2177 - accuracy: 0.3656
Epoch 3/7
30/30 [==============================] - 10s 350ms/step - loss: 1.7055 - accuracy: 0.4729
Epoch 4/7
30/30 [==============================] - 11s 361ms/step - loss: 1.3059 - accuracy: 0.5678
Epoch 5/7
30/30 [==============================] - 11s 360ms/step - loss: 1.0915 - accuracy: 0.6408
Epoch 6/7
30/30 [==============================] - 10s 339ms/step - loss: 0.8861 - accuracy: 0.7208
Epoch 7/7
30/30 [==============================] - 10s 346ms/step - loss: 0.8315 - accuracy: 0.7449


In [14]:
prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
y_pred_zx = prob_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.70      0.74        10
         1.0       0.62      1.00      0.77        10
         2.0       0.50      0.30      0.37        10
         3.0       0.33      0.70      0.45        10
         4.0       0.00      0.00      0.00        10
         5.0       0.60      0.60      0.60        10
         6.0       0.91      1.00      0.95        10
         7.0       0.50      0.80      0.62        10
         8.0       0.91      1.00      0.95        10
         9.0       0.40      0.20      0.27        10
        10.0       1.00      0.70      0.82        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       0.86      0.60      0.71        10
        14.0       0.77      1.00      0.87        10
        15.0       0.00      0.00      0.00        10

    accuracy                           0.54       160
   macro avg       0.51   

In [15]:
y_total = y_pred_xy * y_pred_yz * y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.45      1.00      0.62        10
         2.0       1.00      1.00      1.00        10
         3.0       0.86      0.60      0.71        10
         4.0       0.00      0.00      0.00        10
         5.0       1.00      0.90      0.95        10
         6.0       1.00      0.80      0.89        10
         7.0       0.67      0.80      0.73        10
         8.0       1.00      1.00      1.00        10
         9.0       0.00      0.00      0.00        10
        10.0       0.83      1.00      0.91        10
        11.0       0.00      0.00      0.00        10
        12.0       0.45      0.90      0.60        10
        13.0       0.62      1.00      0.77        10
        14.0       1.00      1.00      1.00        10
        15.0       0.58      0.70      0.64        10

    accuracy                           0.73       160
   macro avg       0.65   

In [16]:
config = '\n\nTEST_USER ' + TEST_USER + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_sptl_bw.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()